# This is PMS data analysis of the videos

Made in environment **PMS_Study**



TO DO: bekijk file PMS_analysis/2_preparation

Probeer veel met loops en functions te werken om de code overzichtelijk te houden

 * Start with file cleanData.csv (geeft informatie over van welke participant welke files bestaan)
 * Load timings.csv file that Tilia and Sofie made
 * Start loop from the top and use Order and A-B
 * In loop check if file (corresponding CSV) exists in Directory (/10001 or /10002 depedent of A/B): later refered to as behaviordata
 * In loop check if file (corresponding .WEBM) exists in Directory
 
If both exist:
- Check if output of Openface exists in Z:\shares\ghepmk_data\2020_Kappen_PMS\10002\video_features (or /10001 dependent of A/B)
- IF True:
Load this as facedata
- IF False:
Print: "Video file present for file (insert filename) but no openface present)"
Break/skip

Use filename/pptnum to search for right starting time in timings.csv variable
- Find timing for beep.wav in behaviordata. 
- Set new zero point (link between the timings.csv point and this)
- Make mean scores on a few variables (we will expand this later) for the corresponding frames of each of the trials.
- To do this, you should write a function that takes the right points of a trial (start, end, trialnum etc) and save this data to a dataframe

Compute this in a way that you would be able to add it to allPMSdata.cvs

Add it all to a new dataframe

Write to csv

UsageError: Line magic function `%magiclb` not found.


In [12]:
%matplotlib inline

In [15]:
import glob
import os
import subprocess
import numpy as np
import pandas as pd
import math

from tqdm import tqdm
from pathlib import Path

import re



In [10]:
# Declare all directories:

data_dir = Path('Z:/shares/ghepmk_data/2020_Kappen_PMS')

# Create all relevant directories
date_dir = "24082021/"
data_all_dir = os.path.join(data_dir, date_dir)
data_A_dir = os.path.join(data_dir, Path('10001'))
data_B_dir = os.path.join(data_dir, Path('10002'))



## uploading the Data
### Testmoment 1 and 2

In [11]:

df = pd.read_csv("Z:/shares/ghepmk_data/2020_Kappen_PMS/24082021/cleanData.csv", encoding = "ISO-8859-1")

A = pd.read_csv("Z:/shares/ghepmk_data/2020_Kappen_PMS/10001/10001")
A.head()




FileNotFoundError: [Errno 2] No such file or directory: 'Z:/shares/ghepmk_data/2020_Kappen_PMS/10001/10001'

In [5]:
from moviepy.editor import VideoFileClip
clip = VideoFileClip("Z:/shares/ghepmk_data/2020_Kappen_PMS/10001/mvideo_{ParticipantNo}_105_25-9-2020.webm")
print( clip.duration )

OSError: MoviePy error: the file Z:/shares/ghepmk_data/2020_Kappen_PMS/10001/mvideo_{ParticipantNo}_105_25-9-2020.webm could not be found!
Please check that you entered the correct path.

In [3]:
pip install moviepy

  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110744 sha256=27a4539cf310d1c6842f888b15fd80b99affb28506ba8261a5d554b4a500d998
  Stored in directory: c:\users\asustek\appdata\local\pip\cache\wheels\e4\a4\db\0368d3a04033da662e13926594b3a8cf1aa4ffeefe570cfac1
  Created wheel for proglog: filename=proglog-0.1.9-py3-none-any.whl size=6157 sha256=0d3e08f29f75f88bf6e496dcf7941c7a29f0d6ced119d69eda6981966c7c4260
  Stored in directory: c:\users\asustek\appdata\local\pip\cache\wheels\22\82\ff\56950e4b223a69eda3a2bdeaef2ae4b274c3d1ced9b5814c99
Successfully built moviepy proglog
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.0
    Uninstalling decorator-5.1.0:
      Successfully uninstalled decorator-5.1.0
Note: you may need to restart the kernel to use updated packages.


In [7]:
import subprocess32 as sp
import json


def probe(vid_file_path):
    ''' Give a json from ffprobe command line

    @vid_file_path : The absolute (full) path of the video file, string.
    '''
    if type(vid_file_path) != str:
        raise Exception('Gvie ffprobe a full file path of the video')
        return

    command = ["ffprobe",
            "-loglevel",  "quiet",
            "-print_format", "json",
             "-show_format",
             "-show_streams",
             vid_file_path
             ]

    pipe = sp.Popen(command, stdout=sp.PIPE, stderr=sp.STDOUT)
    out, err = pipe.communicate()
    return json.loads(out)


def duration(vid_file_path):
    ''' Video's duration in seconds, return a float number
    '''
    _json = probe(vid_file_path)

    if 'format' in _json:
        if 'duration' in _json['format']:
            return float(_json['format']['duration'])

    if 'streams' in _json:
        # commonly stream 0 is the video
        for s in _json['streams']:
            if 'duration' in s:
                return float(s['duration'])

    # if everything didn't happen,
    # we got here because no single 'return' in the above happen.
    raise Exception('I found no duration')
    #return None


if __name__ == "__main__":
    video_file_path = "/tmp/tt1.mp4"
    duration(video_file_path) # 10.008

ModuleNotFoundError: No module named 'subprocess32'

In [8]:
json_info     = probe("Z:/shares/ghepmk_data/2020_Kappen_PMS/10001/mvideo_{ParticipantNo}_105_25-9-2020.webm")
secondes_dot_ = duration("Z:/shares/ghepmk_data/2020_Kappen_PMS/10001/mvideo_{ParticipantNo}_105_25-9-2020.webm") # float number of seconds

NameError: name 'probe' is not defined